In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [ ]:
def data():
    # Split train & validation set
    DDD_Train_Decile5 = pd.read_pickle("DDD_Train_Decile5.pkl")
    DDD_Train_Targets_1D = pd.read_pickle("DDD_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("DDD_Test_Decile5.pkl")
    y_test = pd.read_pickle("DDD_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(DDD_Train_Decile5, DDD_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM DDD Decile 5')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

2264/2264 [==============================] - 8s 3ms/step - loss: 0.4145 - acc: 0.8322 - val_loss: 0.3783 - val_acc: 0.8395
Epoch 36/75
2264/2264 [==============================] - 8s 3ms/step - loss: 0.4196 - acc: 0.8286 - val_loss: 0.3756 - val_acc: 0.8395
Epoch 37/75
2264/2264 [==============================] - 8s 3ms/step - loss: 0.4149 - acc: 0.8295 - val_loss: 0.3737 - val_acc: 0.8430
Epoch 38/75
2264/2264 [==============================] - 8s 3ms/step - loss: 0.4132 - acc: 0.8335 - val_loss: 0.3718 - val_acc: 0.8430
Epoch 39/75
2264/2264 [==============================] - 8s 4ms/step - loss: 0.4090 - acc: 0.8295 - val_loss: 0.3706 - val_acc: 0.8430
Epoch 40/75
2264/2264 [==============================] - 9s 4ms/step - loss: 0.4144 - acc: 0.8308 - val_loss: 0.3696 - val_acc: 0.8430
Epoch 41/75
2264/2264 [==============================] - 7s 3ms/step - loss: 0.4058 - acc: 0.8348 - val_loss: 0.3688 - val_acc: 0.8430
Epoch 42/75
2264/2264 [==============================] - 8s 4ms/ste

2264/2264 [==============================] - 9s 4ms/step - loss: 0.3705 - acc: 0.8366 - val_loss: 0.3358 - val_acc: 0.8554
Epoch 40/75
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3682 - acc: 0.8339 - val_loss: 0.3359 - val_acc: 0.8554
Epoch 41/75
 832/2264 [==========>...................] - ETA: 5s - loss: 0.3347 - acc: 0.8570Train on 2264 samples, validate on 567 samples
Epoch 1/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.6294 - acc: 0.6727 - val_loss: 0.5573 - val_acc: 0.7743
Epoch 2/100
2264/2264 [==============================] - 9s 4ms/step - loss: 0.5312 - acc: 0.7730 - val_loss: 0.4680 - val_acc: 0.7795
Epoch 3/100
2264/2264 [==============================] - 9s 4ms/step - loss: 0.4749 - acc: 0.8057 - val_loss: 0.4191 - val_acc: 0.8254
Epoch 4/100
2264/2264 [==============================] - 9s 4ms/step - loss: 0.4517 - acc: 0.8065 - val_loss: 0.4000 - val_acc: 0.8395
Epoch 5/100
2264/2264 [==============================] - 9s 4

2264/2264 [==============================] - 9s 4ms/step - loss: 0.3882 - acc: 0.8335 - val_loss: 0.3455 - val_acc: 0.8536
Epoch 21/75
2264/2264 [==============================] - 8s 4ms/step - loss: 0.3825 - acc: 0.8317 - val_loss: 0.3443 - val_acc: 0.8536
Epoch 22/75
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3899 - acc: 0.8326 - val_loss: 0.3436 - val_acc: 0.8536
Epoch 23/75
2264/2264 [==============================] - 8s 4ms/step - loss: 0.3854 - acc: 0.8414 - val_loss: 0.3445 - val_acc: 0.8536
Epoch 24/75
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3843 - acc: 0.8344 - val_loss: 0.3434 - val_acc: 0.8554
Epoch 25/75
2264/2264 [==============================] - 8s 3ms/step - loss: 0.3827 - acc: 0.8330 - val_loss: 0.3421 - val_acc: 0.8554
Epoch 63/75
2264/2264 [==============================] - 7s 3ms/step - loss: 0.3814 - acc: 0.8419 - val_loss: 0.3419 - val_acc: 0.8536
Epoch 64/75
2264/2264 [==============================] - 8s 3ms/ste

Epoch 66/125
2264/2264 [==============================] - 7s 3ms/step - loss: 0.3808 - acc: 0.8317 - val_loss: 0.3340 - val_acc: 0.8483
Epoch 67/125
2264/2264 [==============================] - 7s 3ms/step - loss: 0.3814 - acc: 0.8366 - val_loss: 0.3340 - val_acc: 0.8483
Epoch 68/125
2264/2264 [==============================] - 7s 3ms/step - loss: 0.3739 - acc: 0.8357 - val_loss: 0.3342 - val_acc: 0.8483
Epoch 69/125
2264/2264 [==============================] - 7s 3ms/step - loss: 0.3725 - acc: 0.8388 - val_loss: 0.3340 - val_acc: 0.8483
Epoch 70/125
2264/2264 [==============================] - 8s 3ms/step - loss: 0.3755 - acc: 0.8379 - val_loss: 0.3340 - val_acc: 0.8483
Epoch 71/125
2264/2264 [==============================] - 8s 3ms/step - loss: 0.3698 - acc: 0.8366 - val_loss: 0.3339 - val_acc: 0.8483
Epoch 72/125
2264/2264 [==============================] - 7s 3ms/step - loss: 0.3707 - acc: 0.8326 - val_loss: 0.3339 - val_acc: 0.8483
Epoch 73/125
2264/2264 [========================

Epoch 21/50
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3794 - acc: 0.8410 - val_loss: 0.3377 - val_acc: 0.8536
Epoch 22/50
2264/2264 [==============================] - 8s 4ms/step - loss: 0.3704 - acc: 0.8317 - val_loss: 0.3340 - val_acc: 0.8536
Epoch 23/50
2264/2264 [==============================] - 8s 4ms/step - loss: 0.3774 - acc: 0.8379 - val_loss: 0.3385 - val_acc: 0.8571
Epoch 24/50
2264/2264 [==============================] - 7s 3ms/step - loss: 0.3761 - acc: 0.8383 - val_loss: 0.3357 - val_acc: 0.8554
Epoch 25/50
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3686 - acc: 0.8419 - val_loss: 0.3312 - val_acc: 0.8607
Epoch 26/50
2264/2264 [==============================] - 8s 4ms/step - loss: 0.3730 - acc: 0.8335 - val_loss: 0.3331 - val_acc: 0.8519
Epoch 27/50
2264/2264 [==============================] - 8s 4ms/step - loss: 0.3763 - acc: 0.8397 - val_loss: 0.3350 - val_acc: 0.8554
Epoch 28/50
2264/2264 [==============================] 

2264/2264 [==============================] - 8s 4ms/step - loss: 0.4291 - acc: 0.8269 - val_loss: 0.3963 - val_acc: 0.8377
Epoch 30/125
2264/2264 [==============================] - 8s 3ms/step - loss: 0.4324 - acc: 0.8167 - val_loss: 0.3955 - val_acc: 0.8395
Epoch 31/125
2264/2264 [==============================] - 8s 3ms/step - loss: 0.4300 - acc: 0.8216 - val_loss: 0.3932 - val_acc: 0.8413
Epoch 32/125
2264/2264 [==============================] - 8s 4ms/step - loss: 0.4299 - acc: 0.8242 - val_loss: 0.3925 - val_acc: 0.8430
Epoch 33/125
2264/2264 [==============================] - 8s 3ms/step - loss: 0.4254 - acc: 0.8269 - val_loss: 0.3889 - val_acc: 0.8413
Epoch 34/125
2264/2264 [==============================] - 8s 4ms/step - loss: 0.4294 - acc: 0.8238 - val_loss: 0.3886 - val_acc: 0.8448
Epoch 35/125
2264/2264 [==============================] - 8s 4ms/step - loss: 0.4196 - acc: 0.8238 - val_loss: 0.3872 - val_acc: 0.8448
Epoch 36/125
2264/2264 [==============================] - 8s 

Epoch 89/125
2264/2264 [==============================] - 8s 3ms/step - loss: 0.3948 - acc: 0.8304 - val_loss: 0.3531 - val_acc: 0.8519
Epoch 90/125
2264/2264 [==============================] - 8s 4ms/step - loss: 0.3888 - acc: 0.8366 - val_loss: 0.3528 - val_acc: 0.8519
Epoch 91/125
2264/2264 [==============================] - 8s 4ms/step - loss: 0.3884 - acc: 0.8304 - val_loss: 0.3520 - val_acc: 0.8519
Epoch 108/125
2264/2264 [==============================] - 8s 3ms/step - loss: 0.3911 - acc: 0.8339 - val_loss: 0.3522 - val_acc: 0.8519
Epoch 109/125
2264/2264 [==============================] - 8s 3ms/step - loss: 0.3872 - acc: 0.8299 - val_loss: 0.3520 - val_acc: 0.8519
Epoch 110/125
2264/2264 [==============================] - 8s 3ms/step - loss: 0.3965 - acc: 0.8379 - val_loss: 0.3522 - val_acc: 0.8519
Epoch 111/125
2264/2264 [==============================] - 8s 3ms/step - loss: 0.3929 - acc: 0.8330 - val_loss: 0.3521 - val_acc: 0.8519
Epoch 112/125
2264/2264 [===================

2264/2264 [==============================] - 8s 3ms/step - loss: 0.3632 - acc: 0.8361 - val_loss: 0.3369 - val_acc: 0.8519
Epoch 70/100
2264/2264 [==============================] - 8s 4ms/step - loss: 0.3620 - acc: 0.8401 - val_loss: 0.3368 - val_acc: 0.8519
Epoch 71/100
2264/2264 [==============================] - 8s 3ms/step - loss: 0.3694 - acc: 0.8375 - val_loss: 0.3368 - val_acc: 0.8519
Epoch 72/100
2264/2264 [==============================] - 8s 4ms/step - loss: 0.3710 - acc: 0.8414 - val_loss: 0.3368 - val_acc: 0.8519
Epoch 73/100
2264/2264 [==============================] - 8s 4ms/step - loss: 0.3610 - acc: 0.8436 - val_loss: 0.3369 - val_acc: 0.8519
Epoch 74/100
2264/2264 [==============================] - 8s 3ms/step - loss: 0.3664 - acc: 0.8357 - val_loss: 0.3369 - val_acc: 0.8519
Epoch 75/100
2264/2264 [==============================] - 8s 3ms/step - loss: 0.3665 - acc: 0.8366 - val_loss: 0.3369 - val_acc: 0.8519
Epoch 76/100
2264/2264 [==============================] - 8s 

In [ ]:
0.86434